In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('sp500_data.csv')

In [4]:
# take a small sample size to understand what's happening
tickers = ['NVDA', 'GOOG', 'AMZN']
df = data[['ticker', 'year', 'profit_margin', 'p/e', 'eps', 'result']]

bins = [-float('inf'), -1, -0.8, -0.6, -0.4, -0.2, 0, 0.2, 0.4, 0.6, 0.8, 1.0, float('inf')]
labels = ['very low', 'low', 'fairly low', 'moderately low', 'slightly low', 'negative',
          'slightly positive', 'positive', 'fairly high', 'high', 'very high', 'extremely high']

# Create categorical labels
df['category'] = pd.cut(df['result'], bins=bins, labels=labels)

filtered_df = df[data.ticker.isin(tickers)]

C:\Users\Bogdan\AppData\Local\Temp\ipykernel_376\4105595538.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['category'] = pd.cut(df['result'], bins=bins, labels=labels)


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [6]:
X = filtered_df[['profit_margin', 'p/e', 'eps']]
y = filtered_df[['category']]

In [7]:
model = LogisticRegression(max_iter=1000)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
model = model.fit(X_train, y_train)
y_preds = pd.DataFrame(model.predict(X_test))

c:\Users\Bogdan\Desktop\AlgoInvest\.venv\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Bogdan\Desktop\AlgoInvest\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
y_preds = y_preds.rename(columns={0: 'predicted_category'})
y_preds.index = X_test.index
test_df = pd.concat([X_test, y_test, y_preds], axis=1)
merged_df = pd.merge(filtered_df[['ticker', 'year', 'result']], test_df, left_index=True, right_index=True)

In [9]:
merged_df

,ticker,year,result,profit_margin,p/e,eps,category,predicted_category
108,AMZN,2017,0.263965,0.119927,198.814391,5.980503,positive,slightly positive
110,AMZN,2019,0.752453,0.035908,80.870585,22.849346,high,positive
1366,NVDA,2020,1.252548,0.379282,2.935830,20.408759,extremely high,moderately low
1367,NVDA,2021,0.828534,0.182729,4.244891,31.165468,very high,negative


In [10]:
(merged_df.result + 1).prod() - 1

np.float64(8.123438749419863)

In [11]:
merged_df.result

108     0.263965
110     0.752453
1366    1.252548
1367    0.828534
Name: result, dtype: float64